In [1]:
import torch
import sys
sys.path.append('../')
from model.continuous_prompt import ContinuousPromptingLLM
from model.recsys_encoder import RecsysContinuousPromptModel
from model.projection import BasicProjection
from dataset import RecsysDataset

from util import plot_and_save

In [2]:
MODE='train'
TASK='recommendation'
MODEL_NAME = 'light-gcn'
LLM_DIR = "/SSL_NAS/bonbak/model/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3"
SAVE_DIR=f'/home/bonbak/continuous-prompting/output/{TASK}'
TASKS_DIR = f'/home/bonbak/continuous-prompting/task/{TASK}'
DEVICE='cuda:1'

In [3]:
train_dataset = RecsysDataset(f"{TASKS_DIR}/{MODE}.jsonl", f"{TASKS_DIR}/edge.csv")
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
num_users, num_items = len(train_dataset.user_mapping), len(train_dataset.item_mapping)

In [4]:
continuous_prompt_model = RecsysContinuousPromptModel(num_users, num_items,f'{TASKS_DIR}/train_edge_index.pt')
projection_module = BasicProjection(continuous_prompt_model.model.embedding_dim)

model = ContinuousPromptingLLM(
    LLM_DIR,
    continuous_prompt_model, 
    continuous_prompt_model.model.embedding_dim
)

model.continuous_prompt_model.model.load_state_dict(torch.load(f'{SAVE_DIR}/model/{MODEL_NAME}.bin'))

continuous_prompt_model.to(DEVICE)
model.to(DEVICE)

for param in model.parameters():
    param.requires_grad = False
for param in model.projection_module.parameters():
    param.requires_grad = True
for param in model.continuous_prompt_model.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model.train()
c = 0
loss_log_list = []
min_loss = 1000000
accumulate_step = 8

def mean(l):
    return sum(l)/len(l)

for epoch in range(10):
    for input_text, continuous_prompt_input, answer_list in train_dataloader:
        inputs_embeds, attention_mask, labels = model.make_seq2seq_input_label(input_text,continuous_prompt_input,answer_list, embedding_first=True)

        generated_output = model.llm_model.forward(
                    inputs_embeds=inputs_embeds,
                    attention_mask = attention_mask,
                    labels=labels
                )
        generated_output.loss.backward()
        
        if c % accumulate_step == 0:
            optimizer.step()
            optimizer.zero_grad()
        loss_log_list.append(generated_output.loss.item())
        
        if c % 80 == 0 and c!=0:
            cur_loss = mean(loss_log_list[-accumulate_step:])
            if min_loss > cur_loss:
                model.eval()
                model.to('cpu')
                min_loss = cur_loss
                torch.save(model.projection_module.state_dict(), f'{SAVE_DIR}/model/{MODEL_NAME}-projection.bin')
                torch.save(model.continuous_prompt_model.state_dict(), f'{SAVE_DIR}/model/{MODEL_NAME}-encoder.bin')

                inputs_embeds, attention_mask = model.make_input_embed(input_text,continuous_prompt_input, embedding_first=True)
                output = model.llm_model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, pad_token_id=model.llm_tokenizer.eos_token_id, max_new_tokens=1)
                print(input_text[0], model.llm_tokenizer.decode(output[0]))
                plot_and_save(loss_log_list, f'{SAVE_DIR}/loss/{MODEL_NAME}.png')

                model.train()
                model.to(DEVICE)

            print(f'step {c} | cur_loss : {cur_loss:.4f} | min_loss : {min_loss:.4f} ')
        c+=1

사용자 1517의 TV 프로그램 시청 기록:
0. SBS 인기가요
1. 갓파더 케미왕
2. 추석마음한상
3. 뿅뿅 지구오락실
4. 벌거벗은 세계사
5. 정보방송
6. 모범형사 2
7. 추석특집 TV무비 멧돼지사냥
8. 2022 세계육상선수권
9. 씨름의 제왕
10. 참좋은여행 이집트
11. 건강다큐-100세 인생 안녕하십니까?
12. 골 때리는 외박
13. 막사세 - 막내가 사는 세상 2부
14. 2022 FIBA 남자농구 아시아컵
15. 여행의 맛 2부
16. 골프왕3 
17. 찾았다 스트레이 키즈
18. 당신이 소원을 말하면
19. 래오이경제 흑염소즙
20. 열린TV 시청자 세상
21. 서민 갑부
22. 미스터리 듀엣
23. 내 몸을 살리는 발견 유레카 
24. 시사쇼 이것이 정치다
25. 골 때리는 그녀들
26. 울시 골프화
27. 아티스탁 게임 : 가수가 주식이 되는 서바이벌
28. 풍년 아노디끄 IH 압력솥
29. 라스트 모히칸

타겟 TV 프로그램:
* 중계방송 2022년 제1차 정당정책 토론회

질문: 사용자 1517의 TV 프로그램 시청 기록을 고려했을 때, 타겟 TV 프로그램을 사용자가 선호할지 판단해주세요. 반드시 "예" 또는 "아니"로만 대답해야합니다.
답변:  아니
step 80 | cur_loss : 2.8237 | min_loss : 2.8237 
사용자 1471의 TV 프로그램 시청 기록:
0. [가족 특집 3탄] 김구라의 라떼9
1. 도포자락 휘날리며 스페셜
2. 골프왕3 
3. 검색어를 입력하세요 WWW
4. 치얼업
5. 용감한 형사들2
6. 코미디빅리그
7. 비밀남녀
8. 홍김동전
9. 강철부대2 전우회
10. 유 퀴즈 온 더 블록 : 축구에 진심인 자기님들
11. 시그널
12. 글로벌 도네이션쇼 W
13. [MBN 추석특집 영화] 국제수사 
14. WJSN COMEBACK SHOW : SEQUENCE
15. 동상이몽2 너는 내 운명
16. 요즘 육아 금쪽같은 내 새끼 특별판 
17. 돈
18. 이브
19. 도둑들
20. 우리들